### How to access the OpenAI application programming interface

In your command prompt window:
>> pip install openai

https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety



In [52]:
import os
path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

In [53]:
api_key_epl_shared = ""

from openai import OpenAI
# https://stackoverflow.com/questions/36959031/how-to-source-file-into-python-script
execfile(path_equity_precision_llm_repo + "/constants.py")


In [54]:
execfile("epl01_prompt generator.py")
prompt_pmid = prompt_generator(22744164, base_prompts, excel_path)


print(base_prompts[0])
print(base_prompts[1])
print(prompt_pmid)

I am going to outline inclusion criteria for four categories: diabetes, precision medicine, source population, and primary study. 

Please wait for me to prompt you on what to do based on these criteria.  
Here are the inclusion criteria:  

DIABETES: Do not exclude any type of diabetes or prediabetes. The presence of certain conditions or risk factors may not definitively confirm that the study is related to diabetes or prediabetes, unless there is a clear link to diabetes pathophysiology, diagnosis, or management. 

PRECISION MEDICINE: Precision medicine is an assessment of genetic or metabolic state to guide preventive and therapeutic decisions in humans. Exclude epidemiological studies using traditional biomarkers only, focusing on omics (genomics, metabolomics, proteomics, lipidomics etc.) or multi-omics studies. 

SOURCE POPULATION: The source population is correct if the population based on the prompt and the study participantsâ€™ racial/ethnic background, regardless of geograph

In [21]:
# https://platform.openai.com/docs/api-reference/chat/create?lang=python

client = OpenAI(api_key= api_key_epl_shared)

completion = client.chat.completions.create(

    model="gpt-3.5-turbo",

    messages=[

        {"role": "system", "content": base_prompts[0]},
        {"role": "system", "content": base_prompts[1]},
        {"role": "user", "content": prompt_pmid}

    ],

    max_tokens = 1000

)


print(completion.choices[0].message)

ChatCompletionMessage(content='| Article Name | Precision Medicine | Diabetes | Correct Source Population | Primary Study |\n|--------------|-------------------|---------|--------------------------|--------------|\n| Acculturation and glycemic control of Asian Indian adults with type 2 diabetes | No | Yes | South Asia (SA) | Yes |\n\n- **Precision Medicine**: The study does not focus on genetic or metabolic profiling for precision medicine but rather on acculturation and glycemic control, so it does not fulfill the precision medicine criteria.  \n- **Diabetes**: The study clearly focuses on type 2 diabetes, including aspects of diabetes self-management and control, which aligns with the diabetes criteria. \n- **Correct Source Population**: The source population correctly aligns with South Asia (SA) based on the focus on Asian Indian adults in the United States.  \n- **Primary Study**: The study is a mixed-methods study focusing on the association between acculturation and glycemic cont

### 1. Preparing Your Batch File

https://platform.openai.com/docs/guides/batch

.jsonl file where each line contains the details of an individual request to the API

Each request must include a custom_id value

{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}


{"custom_id": "request-2", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are an unhelpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}

In [22]:
pmid_list = pd.read_excel(excel_path, sheet_name='Training Data')['PMID'].tolist()

json_list = []

for index, pmid in enumerate(pmid_list):
    prompt_pmid = prompt_generator(pmid, base_prompts, excel_path)
    dict_pmid = {"custom_id": str(index) + "_" + str(pmid), 
                 "method": "POST", 
                 "url": "/v1/chat/completions", 
                 "body": {"model": "gpt-3.5-turbo-0125", 
                          "messages": [ {"role": "system", "content": base_prompts[0]},
                                        {"role": "system", "content": base_prompts[1]},
                                        {"role": "user", "content": prompt_pmid}],
                            "max_tokens": 1000
                        }
                }

    json_list.append(dict_pmid)



In [23]:
import json
with open(path_equity_precision_llm_folder + '\llm training\Training.jsonl', 'w') as outfile:
    for entry in json_list:
        json_line = json.dumps(entry)
        outfile.write(json_line + '\n')

### 2. Uploading Your Batch Input File

In [26]:
client = OpenAI(api_key= api_key_epl_shared)

batch_input_file = client.files.create(
  file=open(path_equity_precision_llm_folder + '\llm training\Training.jsonl', "rb"),
  purpose="batch"
)

### 3. Creating the Batch

In [27]:
batch_input_file_id = batch_input_file.id
print(batch_input_file_id)
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "training data for PMID query"
    }
)

file-ng31zGIP29yV92yFyHOfmxsB


Batch(id='batch_67376c697964819093df816b601c2be4', completion_window='24h', created_at=1731685481, endpoint='/v1/chat/completions', input_file_id='file-ng31zGIP29yV92yFyHOfmxsB', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1731771881, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'training data for PMID query'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

### 4. Checking the Status of a Batch

In [55]:
client = OpenAI(api_key= api_key_epl_shared)

batch_status = client.batches.retrieve("batch_67376c697964819093df816b601c2be4")
batch_status.output_file_id

'file-yOPAtZ5ZPSxUXPuFHm4fQ8M9'

### 5. Retrieving the Results

In [65]:
client = OpenAI(api_key= api_key_epl_shared)

file_response = client.files.content(batch_status.output_file_id)


In [82]:
import pandas as pd

df = pd.read_json(file_response.content.decode('utf-8'), lines=True)



C:\Users\jvargh7\AppData\Local\Temp\ipykernel_17688\4154174277.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(file_response.content.decode('utf-8'), lines=True)


KeyError: 'body'